First off let's create a couple of useful directories. We will use the template folder in this example.

In [ ]:
import os

if not os.path.exists('templates'):
  os.makedirs('templates')
if not os.path.exists('static'):
  os.makedirs('static')


Now we'll write out an html file and make use of the python jinja library in the process.

In [ ]:
chart = '''
<!DOCTYPE html>
<html lang="en">
<head>
  <title>Chartjs Flask Colab Example</title>
  <canvas id="myChart" width="600" height="500"></canvas>
<script src="https://cdnjs.cloudflare.com/ajax/libs/Chart.js/2.5.0/Chart.min.js"></script>
<script type="text/javascript">

Chart.defaults.global.responsive = false;
 
// defining the chart data
let data = {
  labels : [{% for i in labels %}
             "{{i}}",
            {% endfor %}],
  datasets : [{
      label: '{{ legend }}',
      fill: true,
      data : [{% for j in values %}
                {{j}},
              {% endfor %}]
  }]
}

let ctx = document.getElementById("myChart").getContext("2d");
 
// creating the chart using canvas
let myChart = new Chart(ctx, {
  type: 'line',
  data: data,
});
</script>
</head>
</html>
'''

file=open("templates/chart.html", "w")
file.write(chart)
file.close()


For test data I'm going to use the US Unemployment rate data for 2020 retrieved using the pandas-datareader library.

In [ ]:
import pandas_datareader.data as data
import datetime
import pandas as pd

#US Unemployment Rate
unemp_df = data.DataReader("UNRATE", "fred", datetime.datetime(2020, 1, 1), datetime.datetime(2020, 12, 30))
unemp_df['month'] = unemp_df.index.month_name()
#print(unemp_df)

Time to run Flask and preview with ngrok.

In [ ]:
!pip install flask_ngrok
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template

app = Flask(__name__)
run_with_ngrok(app)
@app.route("/")
def chart():
  legend = "US Unemployment Rate in 2020"
  u_rate = unemp_df['UNRATE'].tolist()
  months = unemp_df['month'].tolist()
  return render_template('chart.html', values=u_rate, labels=months, legend=legend)

app.run()






 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://c9a35345a6c2.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [18/Nov/2020 21:50:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2020 21:50:57] "GET /favicon.ico HTTP/1.1" 404 -
